In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%cd /home/ubuntu/projects/hyper-sloth

In [2]:
from fastcore.all import *
from speedy_utils.all import *
from llm_utils import *

#### Create a LLM model

In [ ]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "01"  # Choose any GPU you want to use
from vllm.lora.request import LoRARequest
from vllm import LLM, SamplingParams
import torch

if "llm" in locals():
    del llm  #
    torch.cuda.empty_cache()
    
llm = LLM(
    model="outputs/lora/Qwen2.5-1.5B-Instruct-LORA-MATH-merge",
    tensor_parallel_size=4,
    task="generate",
    enforce_eager=True,
    dtype=torch.bfloat16,
    max_model_len=16384,
    enable_lora=False,
    # quantization="bitsandbytes", load_format="bitsandbytes",gpu_memory_utilization=0.95
)

tokenizer = llm.get_tokenizer()

from datasets import load_dataset

# Load the GSM8K dataset
gsm8k = load_dataset("gsm8k", "main")
test = gsm8k["test"]


In [4]:
# Prepare prompts for GSM8K evaluation
all_questions = [item["question"] for item in test][:100]
standardized_prompts = [
    tokenizer.apply_chat_template(
        [{"role": "user", "content": f"{question}\nSolve step by step and put your final numerical answer inside \\boxed{{}}"}],
        tokenize=False,
        add_generation_prompt=True,
    )
    for question in all_questions
]

# Set sampling parameters for deterministic generation
sampling_params = SamplingParams(temperature=0.7, top_p=0.95, top_k=64, max_tokens=10000)

# Generate responses for all questions
outputs = llm.generate(standardized_prompts, sampling_params)
all_outputs = [output.outputs[0].text for output in outputs]


Processed prompts: 100%|██████████| 100/100 [01:41<00:00,  1.02s/it, est. speed input: 101.39 toks/s, output: 452.55 toks/s] 


In [5]:
def get_final_output(response):
    try:
        return int(response.split("\\boxed{")[1].split("}")[0])
    except:
        return None

In [27]:
final_outputs = [get_final_output(response) for response in all_outputs]
accs = []
num_error = 0
for i, gt in enumerate(test):
    if i >= len(final_outputs):
        break
    pred = final_outputs[i]
    try:
        num = gt['answer'].split('####')[1]
        num = int(num)
        pred = int(pred)
        accs.append(num == pred)
    except:
        num_error += 1
        accs.append(0)
        pass

In [29]:
np.mean(accs), num_error/len(final_outputs)

(0.59, 0.1)

In [30]:
len(final_outputs)

100